In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict

In [ ]:
# Список русских стоп-слов
RUSSIAN_STOP_WORDS = [
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она',
    'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее',
    'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда',
    'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до',
    'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего',
    'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя',
    'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе',
    'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой',
    'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее',
    'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец',
    'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти',
    'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя',
    'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть',
    'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между'
]

def preprocess_text(text):
    """Предварительная обработка текста"""
    # Приводим к нижнему регистру
    text = text.lower()
    # Удаляем все символы кроме букв и пробелов
    text = ''.join(char for char in text if char.isalnum() or char.isspace())
    return text

def find_duplicates_in_window(df, window_days=2, similarity_threshold=0.6):
    """
    Находит дубликаты новостей в заданном временном окне используя TF-IDF и косинусное сходство
    """
    # Создаем копию датафрейма
    df = df.copy()
    
    # Создаем столбец с datetime и сортируем
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    df = df.sort_values('datetime')
    
    # Предобработка текстов
    df['processed_text'] = df['text'].apply(preprocess_text)
    
    # Создаем и обучаем TF-IDF векторайзер на всем датасете
    vectorizer = TfidfVectorizer(
        min_df=1,
        stop_words=RUSSIAN_STOP_WORDS,  # Теперь передаем список
        ngram_range=(1, 2)  # Добавляем биграммы для лучшего сравнения
    )
    tfidf_matrix_full = vectorizer.fit_transform(df['processed_text'])
    
    # Словарь для хранения групп похожих новостей
    duplicate_groups = defaultdict(set)
    
    # Множество для отслеживания обработанных индексов
    processed_indices = set()
    
    # Проходим по каждой новости
    for i in range(len(df)):
        current_idx = df.index[i]
        
        # Пропускаем, если новость уже в какой-то группе дубликатов
        if current_idx in processed_indices:
            continue
            
        current_date = df.iloc[i]['datetime']
        
        # Определяем временное окно
        date_min = current_date - timedelta(days=window_days)
        date_max = current_date + timedelta(days=window_days)
        
        # Выбираем необработанные новости в окне
        mask_window = (
            (df['datetime'] >= date_min) & 
            (df['datetime'] <= date_max) & 
            ~df.index.isin(processed_indices)
        )
        window_indices = df[mask_window].index
        
        if len(window_indices) > 1:
            # Получаем векторы TF-IDF для текущего окна
            current_idx_in_window = np.where(window_indices == current_idx)[0][0]
            window_tfidf = tfidf_matrix_full[window_indices]
            
            # Вычисляем сходство только для текущего окна
            similarities = cosine_similarity(
                window_tfidf[current_idx_in_window:current_idx_in_window+1], 
                window_tfidf
            )[0]
            
            # Находим похожие новости
            similar_indices = np.where(similarities > similarity_threshold)[0]
            
            if len(similar_indices) > 1:  # Если есть похожие новости
                # Получаем реальные индексы из исходного датафрейма
                similar_indices_full = window_indices[similar_indices]
                
                # Находим самую раннюю новость в группе
                earliest_news = df.loc[similar_indices_full]
                earliest_idx = earliest_news['datetime'].idxmin()
                
                # Добавляем все похожие новости в группу
                duplicate_groups[earliest_idx].update(
                    idx for idx in similar_indices_full if idx != earliest_idx
                )
                
                # Отмечаем все найденные индексы как обработанные
                processed_indices.update(similar_indices_full)
    
    # Собираем все дубликаты в одно множество
    all_duplicates = set()
    for duplicates in duplicate_groups.values():
        all_duplicates.update(duplicates)
    
    # Удаляем найденные дубликаты и ненужные столбцы
    df_cleaned = df.drop(index=list(all_duplicates))
    df_cleaned = df_cleaned.drop(['datetime', 'processed_text'], axis=1)
    
    print(f"Найдено {len(all_duplicates)} дубликатов в {len(duplicate_groups)} группах")
    print(f"Осталось {len(df_cleaned)} уникальных новостей")
    
    # Опционально: выводим пример группы похожих новостей
    if len(duplicate_groups) > 0:
        print("\nПример группы похожих новостей:")
        first_group_key = next(iter(duplicate_groups))
        first_group = [first_group_key] + list(duplicate_groups[first_group_key])
        for idx in first_group:
            print(f"[{df.loc[idx, 'datetime']}] {df.loc[idx, 'text'][:100]}...")
    
    return df_cleaned

In [ ]:
def main():
    # Читаем CSV файл
    df = pd.read_csv('market_impact_news.csv')
    
    # Удаляем дубликаты с временным окном в 2 дня
    df_cleaned = find_duplicates_in_window(df, window_days=2)
    
    # Сохраняем результат
    df_cleaned.to_csv('unique_news.csv', index=False)
    
if __name__ == "__main__":
    main()